In [55]:
from sympy import symbols, integrate, expand, latex

To integrate on the volume of a discrete Affine Body, we can:

1. using volume integral formula directly
2. using divergence theorem to convert the volume integral to a surface integral

We will do both way in this notebook.

# Volume Integral

To integrate on the volume, we need user to provide a tetmesh of the body. With the tetmesh, we can directly calculate the volume of every tetrahedron and sum them up. Say we have $N$ tetrahedrons, the volume of the body is:

$$
V = \sum_i^N V_i = \sum_i^N \int_{\Omega_i} dV
$$

We do integration on each tetrahedron.

For a tetrahedron with vertices $p_0, p_1, p_2, p_3$, we define:

$$
e_1 = p_1 - p_0, e_2 = p_2 - p_0, e_3 = p_3 - p_0
$$

the parameterization of any point $p$ in the tetrahedron is:

$$
p = p_0 + e_1 u + e_2 v + e_3 w, u,v,w \in [0,1], u+v+w \le 1 
$$

The volume of the tetrahedron is:

$$
V_i = \int_{\Omega_i} dV = \int_{\Omega_i} det(e_1, e_2, e_3) du dv dw = det(e_1, e_2, e_3) \int_{\Omega_i} du dv dw
$$

So how to calculate the integral $\int_{\Omega_i} du dv dw$?

In [56]:
u,v,w = symbols('u v w')

fvw = integrate(1, (u, 0, 1-v-w))
fw = integrate(fvw, (v, 0, 1-w))
f = integrate(fw, (w, 0, 1))

f

1/6

$\frac{1}{6}$.

So the volume of the tetrahedron is:

$$
V_i = \frac{1}{6} det(e_1, e_2, e_3) = \frac{1}{6} e_1 \cdot (e_2 \times e_3)
$$

Is just how we calculate the tetrahedron volume geometrically!

So if we have a uniform mass density $\rho$, the mass of the tetrahedron is:

$$
m_i = \rho V_i = \frac{\rho}{6} e_1 \cdot (e_2 \times e_3)
$$

When we calculate the mass matrix of affine body, we not only need the `mass` but also other terms.

$$
(mx^\alpha) = \int_{\Omega} \rho x^\alpha dV = \sum_i^N \int_{\Omega_i} \rho x^\alpha dV, \alpha = 1,2,3
$$

and

$$
(mx^\alpha x^\beta) = \int_{\Omega} \rho x^\alpha x^\beta dV = \sum_i^N \int_{\Omega_i} \rho x^\alpha x^\beta dV, \alpha, \beta = 1,2,3
$$


Now we still using the uniform mass density. $x^\alpha$ is the $\alpha$-th component of the position vector.

So we can integrate on each tetrahedron, consider the parameterization of the tetrahedron, we have:

$$
\begin{align}
\rho \int_{\Omega_i} x^\alpha dV \\
= \rho \int_{\Omega_i} (p_0^\alpha + e_1^\alpha u + e_2^\alpha v + e_3^\alpha w) det(e_1,e_2,e_3) du dv dw \\
= \rho det(e_1,e_2,e_3) \int_{\Omega_i} (p_0^\alpha + e_1^\alpha u + e_2^\alpha v + e_3^\alpha w) du dv dw \\
= \rho det(e_1,e_2,e_3) (p_0^\alpha \int_{\Omega_i} du dv dw + e_1^\alpha \int_{\Omega_i} u du dv dw + e_2^\alpha \int_{\Omega_i} v du dv dw + e_3^\alpha \int_{\Omega_i} w du dv dw)
\end{align}
$$

Something we don't know is the integral terms, we can directly calculate them using sympy.

In [57]:
def integrate_V(f):
    fvw = integrate(f, (u, 0, 1-v-w))
    fw = integrate(fvw, (v, 0, 1-w))
    f = integrate(fw, (w, 0, 1))
    return f

I_dudvdw = integrate_V(1)
I_u_dudvdw = integrate_V(u)
I_v_dudvdw = integrate_V(v)
I_w_dudvdw = integrate_V(w)

I_dudvdw, I_u_dudvdw, I_v_dudvdw, I_w_dudvdw

(1/6, 1/24, 1/24, 1/24)

So we get:

$$
\begin{align}
\int_{\Omega_i} du dv dw = \frac{1}{6}\\
\int_{\Omega_i} u du dv dw = \frac{1}{24}\\
\int_{\Omega_i} v du dv dw = \frac{1}{24}\\
\int_{\Omega_i} w du dv dw = \frac{1}{24}
\end{align}
$$

The rest thing is just apply these coefficients to the corresponding terms.

And let's go to the $(mx^\alpha x^\beta)$ term.

We imagine that we need to calculate the second order integral over $u,v,w$, so just like the first order integral, we can calculate the coefficients of the terms.

It time to expand the $\int_{\Omega_i} \rho x^\alpha x^\beta dV$ term.

$$
\begin{align}
\int_{\Omega_i} \rho x^\alpha x^\beta dV
= \rho det(e_1,e_2,e_3) \int_{\Omega_i}(p_0^\alpha + e_1^\alpha u + e_2^\alpha v + e_3^\alpha w) (p_0^\beta + e_1^\beta u + e_2^\beta v + e_3^\beta w) du dv dw
\end{align}
$$

In [58]:
p_0a, p_0b = symbols('p_0^alpha p_0^beta')
e_1a, e_1b = symbols('e_1^alpha e_1^beta')
e_2a, e_2b = symbols('e_2^alpha e_2^beta')
e_3a, e_3b = symbols('e_3^alpha e_3^beta')

P = expand((p_0a + e_1a * u + e_2a * v + e_3a * w) * (p_0b + e_1b * u + e_2b * v + e_3b * w))

P

e_1^alpha*e_1^beta*u**2 + e_1^alpha*e_2^beta*u*v + e_1^alpha*e_3^beta*u*w + e_1^alpha*p_0^beta*u + e_1^beta*e_2^alpha*u*v + e_1^beta*e_3^alpha*u*w + e_1^beta*p_0^alpha*u + e_2^alpha*e_2^beta*v**2 + e_2^alpha*e_3^beta*v*w + e_2^alpha*p_0^beta*v + e_2^beta*e_3^alpha*v*w + e_2^beta*p_0^alpha*v + e_3^alpha*e_3^beta*w**2 + e_3^alpha*p_0^beta*w + e_3^beta*p_0^alpha*w + p_0^alpha*p_0^beta

$$
\begin{align}

e^{a}_{1} e^{b}_{1} \int_{\Omega_i} u^{2} dudvdw \\
+ e^{a}_{2} e^{b}_{2} \int_{\Omega_i} v^{2} dudvdw\\
+ e^{a}_{3} e^{b}_{3}\int_{\Omega_i} w^{2} dudvdw\\

+ (e^{a}_{1} e^{b}_{2} + e^{b}_{1} e^{a}_{2}) \int_{\Omega_i} u v dudvdw\\
+ (e^{a}_{1} e^{b}_{3} + e^{b}_{1} e^{a}_{3}) \int_{\Omega_i} u w dudvdw\\
+ (e^{a}_{2} e^{b}_{3} + e^{b}_{2} e^{a}_{3}) \int_{\Omega_i} v w dudvdw\\

+ (e^{a}_{1} p^{b}_{0} + e^{b}_{1} p^{a}_{0}) \int_{\Omega_i} u dudvdw\\
+ (e^{a}_{2} p^{b}_{0} + e^{b}_{2} p^{a}_{0}) \int_{\Omega_i} v dudvdw\\
+ (e^{a}_{3} p^{b}_{0} + e^{b}_{3} p^{a}_{0}) \int_{\Omega_i} w dudvdw\\

+ p^{a}_{0} p^{b}_{0} \int_{\Omega_i} dudvdw

\notag
\end{align}
$$

In [59]:
I_uu_dudvdw = integrate_V(u*u)
I_vv_dudvdw = integrate_V(v*v)
I_ww_dudvdw = integrate_V(w*w)

I_uv_dudvdw = integrate_V(u*v)
I_uw_dudvdw = integrate_V(u*w)
I_vw_dudvdw = integrate_V(v*w)

I_uu_dudvdw, I_vv_dudvdw, I_ww_dudvdw, I_uv_dudvdw, I_uw_dudvdw, I_vw_dudvdw

(1/60, 1/60, 1/60, 1/120, 1/120, 1/120)

So we have:

$$
\begin{align}
\int_{\Omega_i} u^2 du dv dw = \frac{1}{60}\\
\int_{\Omega_i} v^2 du dv dw = \frac{1}{60}\\
\int_{\Omega_i} w^2 du dv dw = \frac{1}{60}\\

\int_{\Omega_i} uv du dv dw = \frac{1}{120}\\
\int_{\Omega_i} uw du dv dw = \frac{1}{120}\\
\int_{\Omega_i} vw du dv dw = \frac{1}{120}
\end{align}
$$